In [1]:
# to do: update is_canonical setting in continuous code
# port improved checks for already canonical
# fix special_bigger
# (pretty sure they're all done)

from library import *
from utils import *

uobs = Observable('u', 1)
vobs = Observable('v', 1)
pobs = Observable('p', 0)
#observables = [uobs, vobs, pobs]
observables = [uobs]

### define all parameters and "make library"
N = 6
terms = generate_terms_to(N, observables=observables, max_observables=4)

terms0 = []
terms1 = []
for term in terms:
    if term.rank==0:
        if term.complexity <= N:
            terms0.append(term)
    else:
        if term.complexity <= N:
            terms1.append(term)

In [31]:
x = np.array([[1, 2, 3], [4, 6, 8], [9, 13, 17]])
y = np.dstack([x, 2*x, 4*x])
a = np.array(np.arange(1, 3))
b = np.array(np.arange(1, 3))
c = np.array([0, 2])
ind1 = (a[:, np.newaxis], b)
ind2 = (a[:, np.newaxis, np.newaxis], b[:, np.newaxis], c)
print(a, b)
print(x[ind1])
print(y[ind2])
print(np.multiply.outer(2, np.array([1, 2])))
n_free_dims=0
print(np.expand_dims(a, axis=tuple(range(1, 1+n_free_dims))))

[1 2] [1 2]
[[ 6  8]
 [13 17]]
[[[ 6 24]
  [ 8 32]]

 [[13 52]
  [17 68]]]
[2 4]
[1 2]


In [2]:
print("***Rank 0 library***")
for term in terms0:
    print(term, term.complexity)
print("\n***Rank 1 library***")
for term in terms1:
    print(term, term.complexity)
print(len(terms0), len(terms1))

***Rank 0 library***
1 1
rho 1.33
dj^2 rho 3.33
dj^2 dk^2 rho 5.33
dt rho 2.33
dt dj^2 rho 4.33
dt^2 rho 3.33
dt^2 dj^2 rho 5.33
dt^3 rho 4.33
dt^4 rho 5.33
rho * rho 2.66
rho * dj^2 rho 4.66
rho * dt rho 3.66
rho * dt dj^2 rho 5.66
rho * dt^2 rho 4.66
rho * dt^3 rho 5.66
rho * rho * rho 3.99
rho * rho * dj^2 rho 5.99
rho * rho * dt rho 4.99
rho * rho * dt^2 rho 5.99
rho * rho * rho * rho 5.32
dj rho[u_j] 3
dj^2 dk rho[u_k] 5
dt dj rho[u_j] 4
dt dj^2 dk rho[u_k] 6
dt^2 dj rho[u_j] 5
dt^3 dj rho[u_j] 6
rho * dj rho[u_j] 4.33
rho * dt dj rho[u_j] 5.33
rho * rho * dj rho[u_j] 5.66
rho[u_j * u_j] 3
dj^2 rho[u_k * u_k] 5
dj dk rho[u_j * u_k] 5
dt rho[u_j * u_j] 4
dt dj^2 rho[u_k * u_k] 6
dt dj dk rho[u_j * u_k] 6
dt^2 rho[u_j * u_j] 5
dt^3 rho[u_j * u_j] 6
rho[u_j] * rho[u_j] 4
rho * rho[u_j * u_j] 4.33
dj rho[u_j] * dk rho[u_k] 6
dj rho[u_k] * dj rho[u_k] 6
dj rho[u_k] * dk rho[u_j] 6
rho[u_j] * dj dk rho[u_k] 6
rho[u_j] * dk^2 rho[u_j] 6
rho[u_j] * dt rho[u_j] 5
dt rho * rho[u_j * u_j] 5.

In [4]:
print([x for y in range(1,5) for x in range(1,y)])
print((1,2)+(3,4))

[1, 1, 2, 1, 2, 3]
(1, 2, 3, 4)


In [5]:
cgp = CoarseGrainedPrimitive([uobs, uobs, vobs])
cgp2 = CoarseGrainedPrimitive([uobs, vobs, vobs])
print(cgp)

rho[u * u * v]


In [15]:
print(cgp.index_str([2, 1, 2]))
print(cgp.index_canon([2, 1, 2]))
print(cgp.index_canon([2, 1, 0]))
print(cgp2.index_canon([2, 1, 2]))
print(cgp2.index_canon([2, 1, 0]))

rho[u_k * u_j * v_k]
[1, 2, 2]
[1, 2, 0]
[2, 1, 2]
[2, 0, 1]


In [23]:
labels = ordered_index_list_to_labels([[1, 3, 5], [6, 4, 6], [1, 4, 3], [2, 5, 2]])
print(labels)

{1: [(0, 0), (2, 0)], 3: [(0, 1), (2, 2)], 5: [(0, 2), (3, 1)], 6: [(1, 0), (1, 2)], 4: [(1, 1), (2, 1)], 2: [(3, 0), (3, 2)]}


In [26]:
labels_to_ordered_index_list(labels, [3]*4)

[[1, 3, 5], [6, 4, 6], [1, 4, 3], [2, 5, 2]]

In [3]:
obs_dict = {'u': uobs}
lt = construct_from_string("dj rho[u_i * u_k] @ rho", "LT", obs_dict)
print(lt)
print(lt.structure_canonicalize())
print(lt.obs_list[0] < lt.obs_list[1])
print(lt.obs_list[1] < lt.obs_list[0])
print(sorted(lt.obs_list, reverse=False))
eq = construct_from_string("2 * dj rho[u_i * u_k] @ rho + -1 * rho @ rho", "EQ", obs_dict)
print(eq)

dj rho[u_i * u_k] * rho
rho * dj rho[u_i * u_k]
False
True
[rho, dx rho[u * u]]
2.0 * rho * dj rho[u_i * u_k] + -1.0 * rho * rho = 0
